In [97]:
import numpy as np
import pandas as pd

# data visuzlization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

# warnings
import warnings
warnings.filterwarnings('ignore')

# modeling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score

import joblib
import os
import glob

files = glob.glob('downloaded_models/*')
for f in files:
    os.remove(f)



from google.cloud import storage
import re
# Instantiates a client
storage_client = storage.Client()

# Get GCS bucket
bucket = storage_client.get_bucket('healthcarediabetes')

# Get blobs in bucket (including all subdirectories)
blobs_all = list(bucket.list_blobs())

pre=[x for x in blobs_all if str(x).startswith('<Blob: healthcarediabetes, TRAINED/')]
dates=[re.findall('<Blob: healthcarediabetes, TRAINED/(.*)/.',str(x)) for x in pre]
notnull_Dates=[]
for x in dates:
    for y in x:
        if len(x)>0:
            notnull_Dates.append(y)
dmodel=max(notnull_Dates)


pre=[x for x in blobs_all if str(x).startswith('<Blob: healthcarediabetes, TESTED/')]
dates=[re.findall('<Blob: healthcarediabetes, TESTED/(.*)/.',str(x)) for x in pre]
notnull_Dates=[]
i=0
for x in dates:
    for y in x:
        i=i+1
        if len(x)>0:
            notnull_Dates.append(y)
ddata=max(notnull_Dates)


path="gs://healthcarediabetes/TESTED/"+str(ddata)+'/'+'test_data'
df = pd.read_csv(path)


pre=[x for x in blobs_all if str(x).startswith('<Blob: healthcarediabetes, TRAINED/')]
dates=[re.findall('<Blob: healthcarediabetes, TRAINED/'+str(dmodel)+'/(.*)',str(x)) for x in pre]
notnull_Dates=[]
i=0
MODELS=[]
for x in dates:
    for y in x:
        i=i+1
        if len(x)>0:
            
            MODELS.append(y.split(',')[0])
            
            
if 'readmitted' in df.columns:
    df=df.drop('readmitted',1)
    df=df.iloc[:,1:]

for x in MODELS:
    model_path=r"gs://healthcarediabetes/TRAINED/"+str(dmodel).replace(' ','\ ')+'/'+str(x)
    !gsutil -m cp -r  {model_path}  "downloaded_models"
    model=joblib.load('downloaded_models/'+x)
    
    preds = model.predict(df)



OSError: [Errno 12] Cannot allocate memory

In [ ]:
pre=[x for x in blobs_all if str(x).startswith('<Blob: healthcarediabetes, TRAINED/')]
pre
dates=[re.findall('<Blob: healthcarediabetes, TRAINED/'+str(dmodel)+'/(.*)',str(x)) for x in pre]

In [96]:
df

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,...,Respiratory.1,Circulatory.2,Diabetes.2,Digestive.2,Genitourinary.2,Injury.2,Muscoloskeletal.2,Neoplasms.2,Others.2,Respiratory.2
0,10,3,59,0,18,0,0,0,9,0,...,0,0,0,0,0,0,0,0,1,0
1,20,2,11,5,13,2,0,1,6,0,...,0,0,0,0,0,0,0,0,1,0
2,30,2,44,1,16,0,0,0,7,0,...,0,1,0,0,0,0,0,0,0,0
3,40,1,51,0,8,0,0,0,5,0,...,0,0,1,0,0,0,0,0,0,0
4,50,3,31,6,16,0,0,0,9,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98047,70,3,51,0,16,0,0,0,9,0,...,0,1,0,0,0,0,0,0,0,0
98048,80,5,33,3,18,0,0,1,9,0,...,0,0,0,1,0,0,0,0,0,0
98049,70,1,53,0,9,1,0,0,13,0,...,0,0,0,0,0,0,0,0,1,0
98050,80,10,45,2,21,0,0,1,9,0,...,0,0,0,0,0,1,0,0,0,0
